In [63]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Tesi/Colab working dir'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Tesi/Colab working dir


In [0]:
import tensorflow as tf

### Generator layers
def conv_layer(input, w_size, n_filters, reuse=False, norm='instance', activation='relu', is_training=True, name='conv'):
  """ A w_size x w_size Convolution-ReLU layer with n_filters filters and stride 1
  Args:
    input: 4D tensor
	w_size: integer, window size of the filter (w_size x w_size)
    n_filters: integer, number of filters (output depth)
    norm: 'instance' or 'batch' or None
    activation: 'relu' or 'tanh'
    name: string, e.g. 'conv'
    is_training: boolean or BoolTensor
    name: string
    reuse: boolean
  Returns:
    4D tensor
  """
  with tf.variable_scope(name, reuse=reuse):
    weights = _weights("weights",
      shape=[w_size, w_size, input.get_shape()[3], n_filters])
    h_w_size = int(w_size/2);
    padded = tf.pad(input, [[0,0],[h_w_size,h_w_size],[h_w_size,h_w_size],[0,0]], 'REFLECT')
    conv = tf.nn.conv2d(padded, weights,
        strides=[1, 1, 1, 1], padding='VALID')

    normalized = _norm(conv, is_training, norm)

    if activation == 'relu':
      output = tf.nn.relu(normalized)
    if activation == 'tanh':
      output = tf.nn.tanh(normalized)
    return output

def last_conv_layer(input, w_size, n_filters, reuse=False, name='conv'):
  """ A w_size x w_size Convolution layer with n_filters filters and stride 1
  Args:
    input: 4D tensor
	w_size: integer, window size of the filter (w_size x w_size)
    n_filters: integer, number of filters (output depth)
    name: string, e.g. 'conv'
    name: string
    reuse: boolean
  Returns:
    4D tensor
  """
  with tf.variable_scope(name, reuse=reuse):
    weights = _weights("weights",
      shape=[w_size, w_size, input.get_shape()[3], n_filters])
    h_w_size = int(w_size/2);
    padded = tf.pad(input, [[0,0],[h_w_size,h_w_size],[h_w_size,h_w_size],[0,0]], 'REFLECT')
    output = tf.nn.conv2d(padded, weights,
        strides=[1, 1, 1, 1], padding='VALID')

    return output



def t_conv_layer(input, w_size, n_filters, reuse=False, norm='instance', is_training=True, name=None, output_size=None):
  """ A w_size x w_size fractional-strided-Convolution-BatchNorm-ReLU layer
      with n_filters filters, stride 1/2
  Args:
    input: 4D tensor
    w_size: integer, window size of the filter (w_size x w_size)
    n_filters: integer, number of filters (output depth)
    norm: 'instance' or 'batch' or None
    is_training: boolean or BoolTensor
    reuse: boolean
    name: string, e.g. 't_conv'
    output_size: integer, desired output size of layer
  Returns:
    4D tensor
  """
  with tf.variable_scope(name, reuse=reuse):
    input_shape = input.get_shape().as_list()

    weights = _weights("weights",
      shape=[w_size, w_size, n_filters, input_shape[3]])

    if not output_size:
      output_size = input_shape[1]*2
    output_shape = [input_shape[0], output_size, output_size, n_filters]
    fsconv = tf.nn.conv2d_transpose(input, weights,
        output_shape=output_shape,
        strides=[1, 2, 2, 1], padding='SAME')
    normalized = _norm(fsconv, is_training, norm)
    output = tf.nn.relu(normalized)
    return output


### Helpers
def _weights(name, shape, mean=0.0, stddev=0.02):
  """ Helper to create an initialized Variable
  Args:
    name: name of the variable
    shape: list of ints
    mean: mean of a Gaussian
    stddev: standard deviation of a Gaussian
  Returns:
    A trainable variable
  """
  var = tf.get_variable(
    name, shape,
    initializer=tf.random_normal_initializer(
      mean=mean, stddev=stddev, dtype=tf.float32))
  return var

def _biases(name, shape, constant=0.0):
  """ Helper to create an initialized Bias with constant
  """
  return tf.get_variable(name, shape,
            initializer=tf.constant_initializer(constant))

def _leaky_relu(input, slope):
  return tf.maximum(slope*input, input)

def _norm(input, is_training, norm='instance'):
  """ Use Instance Normalization or Batch Normalization or None
  """
  if norm == 'instance':
    return _instance_norm(input)
  elif norm == 'batch':
    return _batch_norm(input, is_training)
  else:
    return input

def _batch_norm(input, is_training):
  """ Batch Normalization
  """
  with tf.variable_scope("batch_norm"):
    return tf.contrib.layers.batch_norm(input,
                                        decay=0.9,
                                        scale=True,
                                        updates_collections=None,
                                        is_training=is_training)

def _instance_norm(input):
  """ Instance Normalization
  """
  with tf.variable_scope("instance_norm"):
    depth = input.get_shape()[3]
    scale = _weights("scale", [depth], mean=1.0)
    offset = _biases("offset", [depth])
    mean, variance = tf.nn.moments(input, axes=[1,2], keep_dims=True)
    epsilon = 1e-5
    inv = tf.rsqrt(variance + epsilon)
    normalized = (input-mean)*inv
    return scale*normalized + offset

def safe_log(x, eps=1e-12):
  return tf.log(x + eps)


In [0]:
#import tensorflow as tf
#import * from ops
#import utils

class Net:
  def __init__(self, name, n_filters, is_training, norm=None):
    self.name = name
    self.n_filters = n_filters
    self.reuse = False
    self.norm = norm
    self.is_training = is_training

  def __call__(self, input):
    """
    Args:
      input: batch_size x width x height x n_channels
    Returns:
      output: same size as input
    """
    with tf.variable_scope(self.name):
      # conv layers
      conv1 = conv_layer(input, 3, self.n_filters, reuse=self.reuse, norm=self.norm, is_training=self.is_training, name='conv1')
      conv2 = conv_layer(conv1, 3, self.n_filters, reuse=self.reuse, norm=self.norm, is_training=self.is_training, name='conv2')
      conv3 = conv_layer(conv2, 3, self.n_filters, reuse=self.reuse, norm=self.norm, is_training=self.is_training, name='conv3')
      conv4 = conv_layer(conv3, 3, self.n_filters, reuse=self.reuse, norm=self.norm, is_training=self.is_training, name='conv4')
      
      output = last_conv_layer(conv4, 3, 1, reuse=self.reuse, name='output')
      
    # set reuse=True for next call
    self.reuse = True
    self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)

    return output

  def sample(self, input):
    output = self.__call__(input)
    #image = utils.batch_convert2int(self.__call__(input))
    #image = tf.image.encode_jpeg(tf.squeeze(image, [0]))
    return tf.squeeze(output)


In [0]:
#import tensorflow as tf
#import ops
#import utils
#from reader import Reader
#from net import Net

REAL_LABEL = 0.9

class FusionCNN:
  def __init__(self,
               n_filters = 32,
               batch_size=8,
               norm=None,
               learning_rate=2e-4,
               beta1=0.5,
              ):
    """
    Args:
      n_filters: integer, number of filters for each convolutional layer
      batch_size: integer, batch size
      norm: 'None, 'instance' or 'batch'
      learning_rate: float, learning rate for Adam
      beta1: float, momentum term of Adam
    """
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.beta1 = beta1
    self.n_filters = n_filters
    self.norm = norm

    self.is_training = tf.placeholder_with_default(True, shape=[], name='is_training')

    self.net = Net('net', n_filters=self.n_filters, is_training=self.is_training, norm=self.norm)


  def model(self, x, label):
    """
    Args:
      x: ensor of float, sample of the validation set
      label: tensor of float, label related to "x"
    Output:
      loss: float, loss value for the current batch
      fused: tensor of float, output of the CNN for the input "x"
      :return: summaries_training: tf.summary for the current training step
    """
  
    fused = self.net(x)
    loss = self.l2_loss(fused, label)

    # summary
    summaries_training_list = [
                               tf.summary.scalar('training_error',loss)
                              ]
    summaries_training = tf.summary.merge(summaries_training_list)

    return loss, fused, summaries_training

  def validation_summary(self, x, label, mean_loss):
    """
    Args:
      x: tensor of float, sample of the validation set
      label: tensor of float, label related to "x"
      mean_loss: float, validation error in the current epoch
    Output:
      validation_summary: tf.summary, summury for the validation set, images for a sample of the set is saved
    """
    fused = self.net(x)

    x_shape = tf.shape(x)
    x_tof = tf.slice(x, [0, 0, 0, 1], [1, x_shape[1], x_shape[2], 1])
    x_stereo = tf.slice(x, [0, 0, 0, 0], [1, x_shape[1], x_shape[2], 1])
    label_toShow = tf.slice(label, [0, 0, 0, 0], [1, x_shape[1], x_shape[2], 1])
    fused_toShow = tf.slice(fused, [0, 0, 0, 0], [1, x_shape[1], x_shape[2], 1])
    summaries_val_list = [tf.summary.image('val/input_tof', x_tof),
                               tf.summary.image('val/input_stereo', x_stereo),
                               tf.summary.image('val/label', label_toShow),
                               tf.summary.image('val/fused', fused_toShow),

                               tf.summary.scalar('validation_error', mean_loss)
                               ]
    summaries_val = tf.summary.merge(summaries_val_list)

    return summaries_val

  def optimize(self, loss):
    def make_optimizer(loss, variables, name='Adam'):
      """ Adam optimizer with the given learning rate [COMMENTED: for the first 100k steps (~100 epochs)
          and a linearly decaying rate that goes to zero over the next 100k steps]
      """
      
      learning_rate = self.learning_rate
      
      beta1 = self.beta1
      
      global_step = tf.Variable(0, trainable=False)
      """
	  end_learning_rate = 0.0
      start_decay_step = 100000
      decay_steps = 100000
	  learning_rate = (
          tf.where(
                  tf.greater_equal(global_step, start_decay_step),
                  tf.train.polynomial_decay(starter_learning_rate, global_step-start_decay_step,
                                            decay_steps, end_learning_rate,
                                            power=1.0),
                  starter_learning_rate
          )

      )"""
      #tf.summary.scalar('learning_rate/{}'.format(name), learning_rate)

      learning_step = (
          tf.train.AdamOptimizer(learning_rate, beta1=beta1, name=name)
                  .minimize(loss, global_step=global_step, var_list=variables)
      )
      return learning_step

    Net_optimizer = make_optimizer(loss, self.net.variables, name='Adam_net')

    with tf.control_dependencies([Net_optimizer]):
      return tf.no_op(name='optimizers')


  def l2_loss(self,output, label):
    """ loss (MSE norm)
    """
    loss = tf.reduce_mean(tf.squared_difference(label,output))
    return loss
	
  def l1_loss(self,output, label):
    """ loss (MAE norm)
    """
    loss = tf.reduce_mean(tf.absolute_difference(label,output))
    return loss


In [75]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tables
from datetime import datetime
#from dataset import *
#from model import FusionCNN 
from math import ceil


# load the imdb file containing dataset
print("loading dataset")
filepath = 'dataset.mat'
mat_content = tables.open_file(filepath)

training_data = mat_content.root.training_data[:]
training_label = mat_content.root.training_label[:]
validation_full_data = mat_content.root.validation_full_data[:]
validation_full_label = mat_content.root.validation_full_label[:]
test_data = mat_content.root.test_data[:]
test_label = mat_content.root.test_label[:]

print("training samples: {}".format(len(training_data)))
print("validation samples: {}".format(len(validation_full_data)))
print("test samples: {}".format(len(test_data)))

print(training_data.shape)
print(training_label.shape)
print(validation_full_data.shape)
print(validation_full_label.shape)
print(test_data.shape)
print(test_label.shape)

loading dataset
training samples: 3902
validation samples: 8
test samples: 15
(3902, 128, 128, 2)
(3902, 128, 128, 1)
(8, 500, 920, 2)
(8, 500, 920, 1)
(15, 500, 920, 2)
(15, 500, 920, 1)


In [68]:
epoch_num = 100
batch_size = 4
learning_rate = 1e-5
n_filters = 8
early_stopping = True
early_stopping_patience = 4 #number of epochs with no improvement after which training will be stopped.

current_time = datetime.now().strftime("%Y%m%d-%H%M")
checkpoints_dir = "checkpoints/{}".format(current_time)
#--------------------------------------------------------------------------
#------------------------------Graph Structure-----------------------------
#--------------------------------------------------------------------------
print("creating graph")
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, None, None, 2]) #input images
    y = tf.placeholder(tf.float32, shape=[None, None, None, 1]) #output fused disparity map
    mean_val_loss = tf.placeholder(tf.float32)
    
    #initialize dataset
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=2000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat()

    iter = dataset.make_initializable_iterator()
    batch_features, batch_labels = iter.get_next()
    
    cnn = FusionCNN(
                   n_filters = n_filters,
                   batch_size=batch_size,
                   learning_rate=learning_rate)

    loss, fused, summaries_training = cnn.model(batch_features, batch_labels)

    optimizer = cnn.optimize(loss)

    summaries_val = cnn.validation_summary(batch_features, batch_labels, mean_val_loss)

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(checkpoints_dir, graph)
    val_writer = tf.summary.FileWriter(checkpoints_dir, graph,filename_suffix='_Val')

creating graph


In [69]:
file = open('trainingStat_learningRate_'+str(learning_rate)+'_batchSize_'+str(batch_size)+'.txt','a')
config = tf.ConfigProto()
config.gpu_options.allow_growth=True


print("start training")
with tf.Session(graph=graph,config=config) as sess:
  sess.run(tf.global_variables_initializer())
  
  step_counter_glob = 0
  best_val_mse = float("inf")
  stopping_epoch = 0
  train_steps_per_epoch = ceil(len(training_data)/batch_size)
  val_steps_per_epoch = ceil(len(validation_full_data)/batch_size)
  
  for epoch in range(epoch_num):
      train_mse_acc = 0
      
      # initialise iterator with train data
      sess.run(iter.initializer, feed_dict={x: training_data, y: training_label})

      #start epoch
      for step_train in range(train_steps_per_epoch):
          _, loss_val, summary = sess.run([optimizer, loss, summaries_training])
          train_mse_acc += loss_val
          step_counter_glob += 1
          train_writer.add_summary(summary, step_counter_glob)

      # end of epoch: start validation
      validation_mse_acc = 0
      
      # initialise iterator with val data
      sess.run(iter.initializer, feed_dict={x: validation_full_data, y: validation_full_label})
      for step_val in range(val_steps_per_epoch):
          validation_mse_acc += loss.eval()
      
      #calculate mean squared error 
      validation_mse = validation_mse_acc/step_val
      train_mse = train_mse_acc / step_train
      message = 'epoch %d training_mse %g validation_mse %g step_counter %d \n'%(epoch, train_mse, validation_mse, step_train)
      print(message)
      file.write(message)

      # write summaries
      [summaries_val_ev] = sess.run([summaries_val],feed_dict={mean_val_loss: validation_mse})
      val_writer.add_summary(summaries_val_ev, epoch)
      val_writer.flush()
      train_writer.flush()
      
      #early stopping check
      if (validation_mse < best_val_mse):
        stopping_epoch = 0
        best_val_mse = validation_mse
      else:
        stopping_epoch += 1
      if early_stopping and stopping_epoch >= early_stopping_patience:
        print("Early stopping is trigger at epoch: {}".format(epoch))
        break
  
  
  # Save the variables to disk.
  save_path = saver.save(sess, checkpoints_dir + "/model.ckpt")
  print("Model saved in file: %s" % save_path)
  
  # initialise iterator with val data
  sess.run(iter.initializer, feed_dict={x: test_data, y: test_label})
  test_mse_acc = 0
  test_steps_per_epoch = ceil(len(test_data)/batch_size)
  for step_test in range(test_steps_per_epoch):
      test_mse_acc += loss.eval()
  test_mse = test_mse_acc/step_test
  file.write('test error %g \n' % (test_mse))
  file.close()
  val_writer.close()
  train_writer.close()
  

start training
epoch 0 training_mse 1796.46 validation_mse 616.194 step_counter 975 

epoch 1 training_mse 25.9466 validation_mse 74.9415 step_counter 975 

epoch 2 training_mse 11.4036 validation_mse 71.2354 step_counter 975 

epoch 3 training_mse 10.9456 validation_mse 66.5981 step_counter 975 

epoch 4 training_mse 10.2746 validation_mse 57.7132 step_counter 975 

epoch 5 training_mse 9.44942 validation_mse 50.6434 step_counter 975 

epoch 6 training_mse 8.63151 validation_mse 40.7524 step_counter 975 

epoch 7 training_mse 7.87929 validation_mse 31.7911 step_counter 975 

epoch 8 training_mse 7.20991 validation_mse 26.8192 step_counter 975 

epoch 9 training_mse 6.77567 validation_mse 22.087 step_counter 975 

epoch 10 training_mse 6.5269 validation_mse 19.2209 step_counter 975 

epoch 11 training_mse 6.3993 validation_mse 17.369 step_counter 975 

epoch 12 training_mse 6.33182 validation_mse 16.593 step_counter 975 

epoch 13 training_mse 6.29448 validation_mse 15.4748 step_counte

In [0]:
%load_ext tensorboard.notebook
%tensorboard --logdir .

In [71]:
"""
#-----------------------------------------------------------------------
#-----------------------------Show Results------------------------------
#-----------------------------------------------------------------------


  validation_data_shape = validation_full_data.shape
  n = validation_data_shape[0]
  canvas_orig = np.empty((validation_data_shape[0], validation_data_shape[1]))
  canvas_recon = np.empty((validation_data_shape[0], validation_data_shape[1]))
  for i in range(n):
        batch, batch_gt, _ = test_dataset.next_batch(1)#n
        # compute CNN output
        g = sess.run(fused, feed_dict={x_image: batch})

        tof_gt = np.zeros((validation_data_shape[1], validation_data_shape[2]))
        stereo_gt =  np.zeros((validation_data_shape[1], validation_data_shape[2]))
        tof_est =  np.zeros((validation_data_shape[1], validation_data_shape[2]))
        stereo_est = np.zeros((validation_data_shape[1], validation_data_shape[2]))
        # Display GT data
        tof_gt[:,:] = (batch_gt[0,:,:,0])
        stereo_gt[:,:] = (batch_gt[0,:,:,1])
        # Draw the reconstructed data
        tof_est[:,:] = (g[0,:,:,0])
        stereo_est[:,:] = (g[0,:,:,1])

        print("ToF Results")
        plt.figure(figsize=(10, 5))
        plt.subplot(121)
        plot = plt.imshow(tof_gt, origin="upper", cmap="jet", clim=(-0.4, 0.4))
        plt.colorbar()
        plt.title(" ToF Error GT")

        plt.subplot(122)
        plt.imshow(tof_est, origin="upper", cmap="jet", clim=(-0.4, 0.4))
        plt.colorbar()
        plt.title("ToF Estimated Error")
        plt.show()

        print("Stereo Results")
        plt.figure(figsize=(10, 5))
        plt.subplot(121)
        plot = plt.imshow(stereo_gt, origin="upper", cmap="jet", clim=(-0.4, 0.4))
        plt.colorbar()
        plt.title("Stereo Error GT")

        plt.subplot(122)
        plt.imshow(stereo_est, origin="upper", cmap="jet", clim=(-0.4, 0.4))
        plt.colorbar()
        plt.title("Stereo Estimated Error")
        plt.show()
"""

'\n#-----------------------------------------------------------------------\n#-----------------------------Show Results------------------------------\n#-----------------------------------------------------------------------\n\n\n  validation_data_shape = validation_full_data.shape\n  n = validation_data_shape[0]\n  canvas_orig = np.empty((validation_data_shape[0], validation_data_shape[1]))\n  canvas_recon = np.empty((validation_data_shape[0], validation_data_shape[1]))\n  for i in range(n):\n        batch, batch_gt, _ = test_dataset.next_batch(1)#n\n        # compute CNN output\n        g = sess.run(fused, feed_dict={x_image: batch})\n\n        tof_gt = np.zeros((validation_data_shape[1], validation_data_shape[2]))\n        stereo_gt =  np.zeros((validation_data_shape[1], validation_data_shape[2]))\n        tof_est =  np.zeros((validation_data_shape[1], validation_data_shape[2]))\n        stereo_est = np.zeros((validation_data_shape[1], validation_data_shape[2]))\n        # Display GT